# Commands and Variables

In [ ]:
#We will implement the method in the arXiv:1707.07658v2 paper

import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from IPython.display import HTML

from projectq import MainEngine  # import the main compiler engine
from projectq.ops import H, Swap, Tensor, Toffoli, QFT, All, X, Z, Measure, CNOT, StatePreparation, Rx, Ry, Rz, QubitOperator  # import the operations we want to perform (Hadamard and measurement)
from projectq.meta import Control, Dagger, Loop
from projectq.backends import CircuitDrawer

from openfermion.ops import FermionOperator  #the creation annihilation operators
from openfermion.utils import hermitian_conjugated
from openfermion.transforms import jordan_wigner, get_sparse_operator
from openfermion.utils import eigenspectrum, trotterize_exp_qubop_to_qasm as trot

circuit_backend = CircuitDrawer()
#eng = MainEngine(circuit_backend) #uncomment if we want to get a circuit drawing
eng = MainEngine()

N_lattice = 3
precision_eigenvalue = 1
precision_evolution_time = 0.01

# General Functions 


In [ ]:
#Function that writes the decimal value of the binary value with input a list with the binary values
def FractionalBinaryToDecimal(BinaryList):
    DecimalValue = 0.0
    #Since the value of the angle is not exact in base 2, for n bit precision and Prob(correct) = epsilon we need t = n + log( 2 + 1/ 2epsilon). If we take 4 qubits as extra we get an epsilon of 1/28 = 0.036, and taking t = 10 qubits we can get to a good angle close to our wanted value (Either 0.3918265 or 0.67967358), to get a real part of 7/9.
    for i in range(len(BinaryList)):
        if int(BinaryList[-i-1]) == 1:
            DecimalValue += pow(2.0, -(i + 1.0))
    return DecimalValue

# Hamiltonian setup

In this section we will define the Hamiltonian $H = \sum_{i=0}^{i=N} (1-c_{i-1}^\dagger c_{i-1}) c_i^\dagger (1-c_{i+1}^\dagger c_{i+1})$, where $c_{-1}=c_{N+1}=0$

In [ ]:
def CountingOperator(
        i
):  #This creates cdagger c at some position i and takes the boundary conditions into account

    if i == -1:  #Then we are at the first position so there is no left neighbour
        return FermionOperator()  #We return '0' so (1 - n_i) = 1
    if i == N_lattice - 1:  #Then we are at the final lattice point and we have no right noughbour
        return FermionOperator()
    i_string_dagger = str(i) + '^ '  #This creates 'i^ '
    i_string = str(i)  #This creates 'i'
    total_string = i_string_dagger + i_string  #'i^ i'
    c_dagger_c = FermionOperator(total_string)  #This makes cdagger_i c_i

    return c_dagger_c


def Q_dagger_term(i):  #This creates all the individual terms of the sum
    i_dagger = str(i) + '^ '  #'i^ '
    Identity = FermionOperator('')
    term = (Identity - CountingOperator(i - 1)) * FermionOperator(i_dagger) * (
        Identity - CountingOperator(i + 1))  #This is the term we will sum

    return term


Q_dagger = FermionOperator()  #Create empty operator

for i in range(N_lattice):  #Now we create our Operator Q
    Q_dagger += Q_dagger_term(i)

Q = hermitian_conjugated(
    Q_dagger)  #We take the Hermitian Conjugate of this operator

Hamiltonian = Q * Q_dagger + Q_dagger * Q  #This will be our Hamiltonian

# Trotterization setup

We take the Jordan-Wigner transformation of the Hamiltonian. Then we use the openfermion command to generate the trotterized version of this Hamiltonian. This gives us the operator $e^{-iH \delta t}$, for some timestep $\delta t$. This operator applied to a state time evolves the state under the Hamiltonian.

In [ ]:
Hamiltonian_jw = jordan_wigner(
    Hamiltonian)  #We take the jordan-wigner transformation

#This outputs a generator of the code in the QASM language that have to be applied after eachother on the quantum computer
Hamiltonian_trot = trot(
    Hamiltonian_jw, evolution_time=precision_evolution_time)


#Since openfermion generates code in the QASM format I need to change it a bit to get code in the ProjectQ format.
#This command takes as input the lines created by the generator and performs the operations on the qubits in projectq
def QASM_to_ProjectQ(string, qubits):
    if string[:4] == 'CNOT':  #we perform a CNOT
        position_1 = int(string[5])
        position_2 = int(string[7])
        CNOT | (qubits[position_1], qubits[position_2])

    if string[:
              1] == 'H':  #If we read out a hadamard we perform itWe perform the Hadamard
        H | qubits[int(string[2])]

    if string[:1] == 'R':  #We perform a rotation in x,y or z
        #we check where the second ' ' is and take the numbers until there as the angle
        axes = string[1]
        Operation = string[:2]
        angle = float(string[3:string.find(' ', 3)])
        eval(Operation)(angle) | qubits[int(string[-1])]


#This command time evolves some state with the Hamiltonian for some evolution time
def time_evolve(qubits, Hamiltonian_trot):
    for line in Hamiltonian_trot:
        QASM_to_ProjectQ(line, qubits)

# Define the quantum circuit that has the second renyi entropy as eigenvalue

We implement the operator $O = Swap_A(1 - 2|\Psi\rangle \langle \Psi|)$ and we will then use the Quantum Amplitude Estimation algorithm to find the eigenvalue of O namely, $\lambda_\pm = -e^{\pm 2i \theta}$ ,where $\cos(\theta)^2 = \frac{\langle \Psi| Swap_A |\Psi \rangle + 1}{2}$, and $-\frac{1}{2} (\lambda + \bar{\lambda})= \langle \Psi| Swap_A |\Psi \rangle$

In [ ]:
def ProjectorOperator(input_qubit, qureg_1, qureg_2, operator_Q, systemsize_A):
    """Args: (qureg_1 and qureg_2) twice the state of the system and brings the input_qubit into superposition to be used in the QAE algortihm. We also need the eigenvalue of q of the Operator Q with eigenvalue -1, systemsize_A definese on which part we will perform the SWAP operator.
    
    Returns: Superposition of the input_qubit to be used in QAE. Ancillary qubits are used and thrown away in the algorithm.
    """

    #Set up the ancilla qubit to be used in the algorithm for the controlled Swap_A V operator

    q_Ancilla = eng.allocate_qubit(
    )  #Creates 0 state wich by construction will be set to 1 (since the eigenvalue of the operator is -1, its phase is 0.5 which in Binary is 0.1) and then a QFT is applied.

    X | q_Ancilla

    #Fourier transform of the Ancilla qubits
    QFT | q_Ancilla

    #Apply the block of Q^dagger Operators
    with Control(eng, q_Ancilla):
        #We only have one Ancilla qubit and only Q^dagger:
        with Dagger(eng):
            operator_Q(qureg_1, qureg_2)

    #Apply a Hadamard on the ancillary qubit
    H | q_Ancilla

    #To control on the |0> state we apply an X on all qubits. Then control on the |1>. Then apply an X again.
    X | q_Ancilla
    with Control(eng, q_Ancilla):
        Z | input_qubit
    X | q_Ancilla

    #Apply a Hadamard on the ancillary qubit
    H | q_Ancilla

    #Apply the block of Q operators
    #We apply Operator Q on both systems together
    with Control(eng, q_Ancilla):
        operator_Q(qureg_1, qureg_2)

    #Control SWAP on the input_qubit
    with Control(eng, input_qubit):
        for i in range(systemsize_A):
            Swap | (qureg_1[i], qureg_2[i])

    #Bring the Ancilla back to the computational basis, measure and deallocate
    with Dagger(eng):
        QFT | q_Ancilla

    Measure | q_Ancilla
    del q_Ancilla

# Quantum Phase Estimation

In [ ]:
#Now we write our Quantum Amplitude Amplification algorithm, where we do a phase estimation to the operator we just created.
def QuantumAmplitude(n_precision, qureg_1, qureg_2, operator_Q, systemsize_A):

    #Set up phase estimation algorithm
    #We set up the desired 0 qubits
    qureg_0 = eng.allocate_qureg(n_precision)

    #Do a QFT (or the same a Hadamard on every qubit) on these qubits
    QFT | qureg_0

    for i in range(
            n_precision
    ):  #loop over the qubits and apply the operator 2**i times for the phase estimation algorithm
        with Loop(
                eng, 2**i
        ):  #We don't need to control on the qubits since this is already done in the operator algorithm. The Operator is performed controlled 2**(n_precision - 1) times on the first qubit and 2**0 times on the last qubit. This way the first qubit we measure will be the first qubit of the base 2 expansion of the phase.
            ProjectorOperator(qureg_0[-i-1], qureg_1, qureg_2, operator_Q,
                              systemsize_A)
#         print('Done the Controlled Operator %i' % (2**i))

    #Do the QFT^dagger on qureg_0
    with Dagger(eng):
        QFT | qureg_0

    #Measure all qubits
    All(Measure) | qureg_0

    #Phase estimation gives us the angle of the eigenvalue.
    Phi_Estimation = FractionalBinaryToDecimal(qureg_0)
    
    for i in range(len(qureg_0)):
#         print('Value of %i qubit %i' % (i, int(qureg_0[i])))

    print('Estimated angle is  %f' % Phi_Estimation)
    ExpTheta = np.exp(2.0 * np.pi * 1.0j * Phi_Estimation)
    # #print(ExpTheta)
    # ExpThetaConj = np.conjugate(ExpTheta)
    # #print(ExpThetaConj)

    # R2 = - (ExpTheta + ExpThetaConj) / 2

    #Minus the real part of ExpTheta
    R2 = -np.real(ExpTheta)
    del qureg_0
    
    return R2

# Initial State Preparation and desired Operator

This function makes from the all zero state the state $\frac{1}{\sqrt{3}}(|01\rangle + |10 \rangle + |11\rangle$). With this function we make the operator that adds a minus to our state of interest and leaves all other states untouched, thus it sends $|\Phi \rangle \to |\Phi \rangle$ except for our state $|\Psi \rangle \to -|\Psi \rangle$.

In [ ]:
def Generate_3_Bell_States_Superposition(q2):
    #Bring the state from |00> -> 1/sqrt3 |00> + sqrt2/sqrt3 |10>, the factor of two comes in from the fact that rotations around Y have the angle divided by 2.
    Ry( 2.0 * np.arccos( 3**(-0.5) ) ) | q2[0]
    
    #Now we do a controlled Hadamard to go to 1/sqrt3 (|00> + |10> + |11> )
    with Control(eng, q2[0]):
        H | q2[1]
    
    #Now one would want to bring the |00> state to |01> this can be done as follows
    """
    X | q2[0]
    with Control(eng, q2[0]):
        X | q2[1]
    X | q2[0]
    """
    
    #But in this case only an X gate on the second qubit would already work
    #1/sqrt3 (|00> + |10> + |11> ) -> 1/sqrt3( |01> + |11> + |10> )
    X | q2[1]
    
def Controlled_Q_3qubits(qureg0, qureg1):
    #Bring to all zero
    with Dagger(eng):
        Generate_3_Bell_States_Superposition(qureg0)
    with Dagger(eng):
        Generate_3_Bell_States_Superposition(qureg1)
    
    #Take both registers together to perform the control
    qureg2 = qureg0 + qureg1 
    
    All(X) | qureg2
    
    with Control(eng, qureg2[1:len(qureg2)]):
        Z | qureg2[0]
    
    All(X) | qureg2
    
    #Bring to proper state again
    Generate_3_Bell_States_Superposition(qureg0)
    Generate_3_Bell_States_Superposition(qureg1)

# Main

In [ ]:
result_lst = []

for i in tqdm(range(1,12)):
    qureg1 = eng.allocate_qureg(2)
    qureg2 = eng.allocate_qureg(2)


    #Bring the two states into superposition.
    Generate_3_Bell_States_Superposition(qureg1)
    Generate_3_Bell_States_Superposition(qureg2)

    Result = QuantumAmplitude(i, qureg1, qureg2, Controlled_Q_3qubits, 1)
    result_lst.append(Result)
    
    All(Measure) | qureg1
    All(Measure) | qureg2
    eng.flush() 

print('Minus the real part of the eigenalue is %f' % Result)
Renyi2 = -np.log(Result) / np.log(2)
print('Second Renyi entropy is %f' % Renyi2)

In [ ]:
result_lst

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.ylim([0.5,1.1])
plt.plot(np.linspace(1,12,11), result_lst, label = 'Simulation')
plt.plot(np.linspace(1,12,11), [ 7/9 for i in range(11)], label = 'Exact Value')
plt.xlabel('Number of ancilla qubits')
plt.ylabel(r'Tr[ $\rho_A^2$ ]')
plt.legend()
plt.title(r'Estimation of Tr[ $\rho_A^2$ ] using the Quantum Amplitude Estimation procedure.')
plt.show()